In [ ]:
from google.colab import drive
import joblib

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content


!tar -xvzf /content/drive/MyDrive/audio_mag/cv-corpus-10.0-delta-2022-07-04-ru.tar.gz


Выходные данные были обрезаны до нескольких последних строк (5000).
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32033953.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32033961.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32033967.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034295.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034299.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034303.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034313.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034322.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034563.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034569.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034575.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034580.mp3
cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32034583.mp3
cv-corpus-10.0-delta-2

In [ ]:
pwd

'/content'

In [ ]:
import pandas as pd


df = pd.read_csv('/content/cv-corpus-10.0-delta-2022-07-04/ru/other.tsv', sep='\t')


df = df[df['age'].isin(['teens', 'twenties', 'thirties', 'fourties', 'fifties', 'sixties'])]

print(df['path'])

df['file_path'] = '/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/' + df['path']


5       common_voice_ru_31852309.mp3
6       common_voice_ru_31852310.mp3
7       common_voice_ru_31852311.mp3
8       common_voice_ru_31852313.mp3
9       common_voice_ru_31852314.mp3
                    ...             
4378    common_voice_ru_32941266.mp3
4379    common_voice_ru_32941267.mp3
4380    common_voice_ru_32941268.mp3
4381    common_voice_ru_32941269.mp3
4382    common_voice_ru_32941270.mp3
Name: path, Length: 2588, dtype: object


In [ ]:
import librosa
import numpy as np
import os

def extract_features(file_path):
    print(file_path)
    if not os.path.exists(file_path):
        print(f"Файл не найден: {file_path}")
        return None

    try:
        y, sr = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Ошибка при обработке {file_path}: {e}")
        return None

df = df.sample(1000, random_state=42).reset_index(drop=True)
X = np.array([extract_features(fp) for fp in df['file_path']])
y = df['age'].values

print(df.shape)

/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32316642.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32316594.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32941250.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32182460.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32263610.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32065304.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32127964.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32420935.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32862588.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32065250.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32271678.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_32703791.mp3
/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/co

In [ ]:
print(X)

[[-332.39215    108.52303     16.236269  ...   -9.635975    -7.46507
   -14.663033 ]
 [-378.67212     84.73231     11.774603  ...   -4.5938454   -7.3500113
   -10.626081 ]
 [-427.72266     54.934345   -25.348783  ...   -2.3402817   -2.8395538
   -10.601821 ]
 ...
 [-321.80542     84.29021    -19.56651   ...   -4.540785     2.308001
   -23.361343 ]
 [-442.6721      49.168938     4.498434  ...   -2.265804    -7.322623
     4.1839285]
 [-400.64908     54.804955    -7.1317196 ...  -12.731019   -11.969995
    -7.8875523]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    fourties       0.96      0.85      0.91        62
       teens       0.90      0.99      0.94        95
    thirties       1.00      0.71      0.83        17
    twenties       0.86      0.92      0.89        26

    accuracy                           0.92       200
   macro avg       0.93      0.87      0.89       200
weighted avg       0.92      0.92      0.91       200



In [ ]:
file = '/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_31858694.mp3'
features = extract_features(file).reshape(1, -1)
print("Предсказанный возраст:", clf.predict(features)[0])


/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_31858694.mp3
Предсказанный возраст: fourties


In [ ]:
joblib.dump(clf, 'age_classifier.pkl')

['age_classifier.pkl']

In [ ]:
clf = joblib.load('age_classifier.pkl')
file_path = '/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_31858694.mp3'
features = extract_features(file_path).reshape(1, -1)
predicted_age = clf.predict(features)[0]
print("Предсказанный возраст:", predicted_age)

/content/cv-corpus-10.0-delta-2022-07-04/ru/clips/common_voice_ru_31858694.mp3
Предсказанный возраст: fourties
